## Building Footprint Data
Loading bounding coordinates of buildings in the area.

In [3]:
import xml.etree.ElementTree as ET

def read_kml(file_path):
    # Parse the KML file
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    # Define the namespace
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    
    # Find all Placemarks
    placemarks = root.findall('.//kml:Placemark', ns)
    print(f"Found {len(placemarks)} placemarks")

# Example usage
kml_file = "../data/Building_Footprint.kml"  # Make sure this path is correct
read_kml(kml_file)

Found 9436 placemarks


In [4]:
import xml.etree.ElementTree as ET
import folium

def parse_coordinates(coord_str):
    # Split coordinates into pairs and convert to float
    coords = []
    pairs = coord_str.strip().split(' ')
    for pair in pairs:
        if pair:
            lon, lat = pair.split(',')[:2]  # Take only first two values
            coords.append([float(lat), float(lon)])  # Folium uses [lat, lon]
    return coords

def read_and_plot_kml(file_path):
    # Parse KML
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    # Create map centered on approximate middle of NYC
    m = folium.Map(location=[40.785, -73.95], zoom_start=13)
    
    # Define namespace
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}
    
    # Find all coordinate sets
    for coords in root.findall('.//kml:coordinates', ns):
        coord_list = parse_coordinates(coords.text)
        if coord_list:
            # Create polygon for each building
            folium.Polygon(
                locations=coord_list,
                color='blue',
                fill=True,
                weight=1
            ).add_to(m)
    
    # Save map
    m.save('building_footprints.html')

# Use the function
kml_file = "../data/Building_Footprint.kml"  # Adjust path as needed
read_and_plot_kml(kml_file)

## NY Mesonet Weather Data
### Background

This spreadsheet contains detailed weather data provided by the New York State Mesonet. There are two datasets (see worksheet tabs) located in the Bronx (Latitude=40.87248, Longitude=-73.89352, Altitude=57.5 meters) and Manhattan (Latitude = 40.76754, Longitude = -73.96449, Altitude = 94.8 meters). These two weather stations are located at the southern and northern ends of the ground traverse data (see "Location" tab). The data was collected near the surface (2-meters height) every 5 minutes during the day. This data may be useful for your AI model as it contains air temperature, wind speed and direction, and solar isolation (solar flux at the surface). 

### How might you use this data?

* Temperature and Relative Humidity - These data were collected by the sensor at 2-meters height, near the ground. Since these data points are not part of the core challenge dataset, they should only be used for reference. As a note, when temperature is combined with relative humidity, it yields a "heat index" which is the apparent temperature experienced by the human body. High values of "heat index" have been known to contribute to health issues. 

* Wind Speed and Direction - It may be possible to use this information in your model. For example, wind speed and direction may impact the local mixing of air at the surface depending on local building geometries. Such variations in air mixing could impact local urban heating conditions. 

* Solar Flux - It may be possible to use this information in your model. For example, solar flux is the amount of solar energy that reaches the ground which is impacted by cloud cover and building shading. When combined with building footprint, building height, and sun angle data, it may be possible to predict the shading conditions for specific locations on the ground. Such information could impact local urban heating conditions. 

### How was temperature data collected for this challenge?

The core temperature data for this challenge was collected using ground traverses in cars. The data was collected in the afternoon from 3:00pm to 4:00pm. The data was then converted to a UHI index using the actual temperature at any location divided by the mean temperature of the dataset. 





In [5]:
import pandas as pd

def load_weather_data(file_path):
    # Read specific sheets
    bronx_df = pd.read_excel(file_path, sheet_name='Bronx')
    manhattan_df = pd.read_excel(file_path, sheet_name='Manhattan')
    
    # Convert Date/Time column to datetime
    bronx_df['Date / Time'] = pd.to_datetime(bronx_df['Date / Time'])
    manhattan_df['Date / Time'] = pd.to_datetime(manhattan_df['Date / Time'])
    
    # Display basic info about each dataframe
    print("Bronx DataFrame:")
    print(bronx_df.head())
    print("\nShape:", bronx_df.shape)
    
    print("\nManhattan DataFrame:")
    print(manhattan_df.head())
    print("\nShape:", manhattan_df.shape)
    
    return bronx_df, manhattan_df

# Load the data
excel_file = "../data/NY_Mesonet_Weather.xlsx" 
bronx_df, manhattan_df = load_weather_data(excel_file)

Bronx DataFrame:
          Date / Time  Air Temp at Surface [degC]  \
0 2021-07-24 06:00:00                        19.3   
1 2021-07-24 06:05:00                        19.4   
2 2021-07-24 06:10:00                        19.3   
3 2021-07-24 06:15:00                        19.4   
4 2021-07-24 06:20:00                        19.4   

   Relative Humidity [percent]  Avg Wind Speed [m/s]  \
0                         88.2                   0.8   
1                         87.9                   0.8   
2                         87.6                   0.7   
3                         87.4                   0.5   
4                         87.0                   0.2   

   Wind Direction [degrees]  Solar Flux [W/m^2]  
0                       335                  12  
1                       329                  18  
2                       321                  25  
3                       307                  33  
4                       301                  42  

Shape: (169, 6)

Manhattan

/var/folders/fg/gtyh9bm95ssfs2_k2xlfk2d80000gp/T/ipykernel_33795/4112188364.py:9: FutureWarning: Parsed string "2021-07-24 06:00:00 EDT" included an un-recognized timezone "EDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  bronx_df['Date / Time'] = pd.to_datetime(bronx_df['Date / Time'])
/var/folders/fg/gtyh9bm95ssfs2_k2xlfk2d80000gp/T/ipykernel_33795/4112188364.py:10: FutureWarning: Parsed string "2021-07-24 06:00:00 EDT" included an un-recognized timezone "EDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  manhattan_df['Date / Time'] = pd.to_datetime(manhattan_df['Date / Time'])
